Code to utilize trained model. A user will require an open ai key with openai api access (currently not free) and access to the pinecone.ai repository.

In [ ]:
#install libraries required
!pip install python-dotenv
!pip install openai
!pip install langchain_openai
!pip install -U langchain-cli
!pip install -qU pinecone-client==3.1.0 pandas==2.0.3
!pip install langchain
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.7/113.7 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#library imports
from pinecone import Pinecone
from google.colab import userdata
from dotenv import load_dotenv
import os

# to tokenize, chunk, and embed the text
from openai import OpenAI
import nltk
import tiktoken
from typing import List


# to distill the results
from langchain_openai import ChatOpenAI
from langchain.docstore.document import Document
from langchain.chains.question_answering import load_qa_chain

#for user interface:
import gradio as gr

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#get access to pinecone and openai
pc = Pinecone(api_key=userdata.get('PINECONE_API_KEY'))
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
index = pc.Index('agriculture-project')

In [68]:
# Setup the models and essential files needed for creating chat output
OPENAI_MODEL = "gpt-3.5-turbo"
EMBED_MODEL = "text-embedding-3-small"
# Store the API key in a variable.

client = OpenAI(api_key=OPENAI_API_KEY)
MAX_TOKENS = 1536

def prep(text: str):
    return text.replace("\n", " ").replace("\r", " ").replace("\t", " ")

def tokenize(text: List[str]):
    encoding = tiktoken.encoding_for_model(EMBED_MODEL)
    return encoding.encode(text)

def embed(tokens: List[int]):
    response = client.embeddings.create(input=tokens,model=EMBED_MODEL)
    return response.data[0].embedding

def chunk_text(text:str):
    current_chunk = []
    current_para = ""
    chunks = []
    paras = []
    current_len = 0
    sentences = nltk.sent_tokenize(text)
    chunks_of_tokens = []

    for sentence in sentences:
        # Tokenize the sentence
        sentence_tokens = tokenize(sentence)
        sentence_token_len = len(sentence_tokens)

        # Check if adding the next sentence exceeds the max token limit
        if current_len + sentence_token_len > MAX_TOKENS:
            # Add the current chunk to the list and start a new one
            paras.append(current_para)
            current_para = ""
            chunks_of_tokens.append(current_chunk)
            embeddings = embed(current_chunk)
            chunks.append(embeddings)
            current_chunk = []
            current_len = 0

        # Add the sentence to the current chunk
        current_para += " " + sentence
        current_chunk.extend(sentence_tokens)
        current_len += sentence_token_len

    # Add the last chunk if it's not empty
    if current_chunk:
        paras.append(current_para)
        chunks_of_tokens.append(current_chunk)
        embeddings = embed(current_chunk)
        chunks.append(embeddings)

    return paras, chunks, chunks_of_tokens

def create_embeddings(filename: str):
    with open(filename, "r") as file:
        text = file.read()
    text = prep(text)
    return chunk_text(text)

def create_embeddings_prompt(prompt:str):
    prompt = prep(prompt)
    return chunk_text(prompt)

def vectorize_chunks(paras: List, chunks: List, **kwargs):
    vectors = []
    for i in range(len(chunks)):
        if "filename" in kwargs:
            vectors.append({"id": f"{i}", "values": chunks[i], "metadata": {"file": filename, "para": f"{paras[i]}"}})
        else:
            vectors.append({"id": f"{i}", "values": chunks[i], "metadata": {"para": f"{paras[i]}"}})

    return vectors

In [69]:
#create function for taking user questions
query_responses=[]

def ask_a_question(prompt):
    # convert the prompt to chunks of  embeddings
    paras, chunks, chunks_of_tokens  = create_embeddings_prompt(prompt)
    #print(f"Embeddings: {chunks[0]}")
    # vectorize the embeddings
    prompt_vectors = vectorize_chunks(paras, chunks)
    #print(f"Vectorized: {prompt_vectors[0]}")
    # search the index for the best match using semantic search
    query_response = index.query(
        top_k=2,
        vector=prompt_vectors[0]["values"]
    )
    query_responses.append(query_response)
    #print(f"Query response: {query_response}")
    # get the id of the best match
    best_id = query_response["matches"][0]["id"]
    #print(f"Best ID: {best_id}")
    # fetch the best match from the index
    result = index.fetch(ids=[best_id])
    # get the paragraph of interest from the result metadata
    para_of_interest = result["vectors"][best_id]["metadata"]["para"]
    #print(f"Para of interest: {para_of_interest}")
    # Initialize the langchain chat model.
    llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, temperature=0.0)
    # turn the para_of_interest into a Document
    document = Document(page_content=para_of_interest)
    # Create the QA chain using the LLM.
    chain = load_qa_chain(llm)
    # Pass the para_of_interest and the prompt to the chain, and print the result.
    #question = "If you can't find the answer in the provided document, say, I just don't know the answer to that, otherwise, answer the question. " + prompt
    question = prompt
    result = chain.invoke({"input_documents": [document], "question": question})
    return result["output_text"]

In [72]:
#create questions to be asked
query_responses=[]

questions = ["how many eclipses will there be this year?",
            "will there be four eclipses in 2024?",
            "when are the solar eclipses?",
            "when are the lunar eclipses?",
            "what is the date of the solar eclipse 2024?",
            "what is the most popular farm animal in the United States?",
            "what information do you have on duck eggs?",
            "can you summarize page 201 of the almanac?",
            "summarize astronomy eclipses",
            "tell me what a farmer should know about farming in 2024."
]

answers = []
for question in questions:
    answers.append(ask_a_question(question))

In [73]:
#generate the answers
ix = 0
for query_response in query_responses:
    print(f"Match Score: {query_response['matches'][0]['score']}")
    print(f"Question: {questions[ix]}")
    print(f"Answer:   {answers[ix]}\n\n")
    ix += 1

Match Score: 0.524506092
Question: how many eclipses will there be this year?
Answer:   There were four lunar eclipses in 2020, all of the penumbral variety.


Match Score: 0.488794893
Question: will there be four eclipses in 2024?
Answer:   I don't know.


Match Score: 0.594897628
Question: when are the solar eclipses?
Answer:   Solar eclipses are rare events that occur sporadically. Any given place on Earth sees a solar totality only once every 360 years, on average. The next total solar eclipse in Los Angeles, for example, will not occur until 3290. The randomness of the universe means that relatively few people have seen a total solar eclipse.


Match Score: 0.538452327
Question: when are the lunar eclipses?
Answer:   There will be a penumbral lunar eclipse on March 24-25, 2024.


Match Score: 0.550877869
Question: what is the date of the solar eclipse 2024?
Answer:   The total solar eclipse of 2024 will occur on April 8th.


Match Score: 0.43765232
Question: what is the most popul

In [74]:
question_input = gr.Textbox(label="Ask a Question to the Farmer's Almanac")
iface = gr.Interface(
    fn=ask_a_question,
    inputs=question_input,
    outputs="text",  # We expect a text output
    title="Question Answering GUI",
    description="Ask a question and get an answer.",
)
iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://6735a76cb97eccdf37.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
